# MLiM: Purchase Prediction - Recommender System

* predict purchase probabilities for all customer-product combinations
* time period for making predictions is week 90
* 2000 shoppers and 250 products -> 500k predictions

In [2]:
import os
import tqdm
import warnings
import functools

import numpy as np
import pandas as pd

import seaborn as sns
import sklearn.preprocessing
import sklearn.neighbors
import sklearn.model_selection
import sklearn.metrics

import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns
color = sns.color_palette()
%matplotlib inline
pd.options.mode.chained_assignment = None  # default='warn'

import pyarrow
import fastparquet

In [4]:
os.getcwd()

'/Users/asmir/mlim_project/mlim/exercises/ex3'

In [57]:
from statsmodels.graphics.tsaplots import plot_acf

In [ ]:
#plot_acf(data['Count'], lags=10)
#plot_pacf(data['Count'], lags=10)

In [5]:
baskets = pd.read_parquet("baskets.parquet")

In [6]:
baskets.head() # 68.8mm products # 99999 shoppers # week 0-89

,week,shopper,product,price
0,0,0,71,629
1,0,0,91,605
2,0,0,116,715
3,0,0,123,483
4,0,0,157,592


In [7]:
pred_idx = pd.read_parquet("prediction_index.parquet")

In [8]:
pred_idx.head() # 2000 shopppers: 0-1999, 500.000 shopper-product combinations (predictions)

,week,shopper,product
0,90,0,0
1,90,0,1
2,90,0,2
3,90,0,3
4,90,0,4


In [12]:
coupons = pd.read_parquet("coupons.parquet")

In [13]:
coupons.head() # 45mm products discounted # 99999 shoppers # week 0-89

,week,shopper,product,discount
0,0,0,35,35
1,0,0,193,40
2,0,0,27,30
3,0,0,177,35
4,0,0,5,30


In [29]:
data = pd.merge(baskets, coupons, on=['week','shopper','product'], how='left')
data.head()

,week,shopper,product,price,discount
0,0,0,71,629,NaN
1,0,0,91,605,NaN
2,0,0,116,715,NaN
3,0,0,123,483,NaN
4,0,0,157,592,NaN


In [31]:
data['discount'] = data['discount'].replace(np.nan, 0)

In [32]:
data.head()

,week,shopper,product,price,discount
0,0,0,71,629,0.0
1,0,0,91,605,0.0
2,0,0,116,715,0.0
3,0,0,123,483,0.0
4,0,0,157,592,0.0


In [46]:
data_2 = data[data['shopper'] < 2000]
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1378720 entries, 0 to 68091873
Data columns (total 5 columns):
 #   Column    Non-Null Count    Dtype  
---  ------    --------------    -----  
 0   week      1378720 non-null  int64  
 1   shopper   1378720 non-null  int64  
 2   product   1378720 non-null  int64  
 3   price     1378720 non-null  int64  
 4   discount  1378720 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 63.1 MB


In [47]:
data_2.head()# 1.38mm 

,week,shopper,product,price,discount
0,0,0,71,629,0.0
1,0,0,91,605,0.0
2,0,0,116,715,0.0
3,0,0,123,483,0.0
4,0,0,157,592,0.0


In [49]:
#clearing memory 
del baskets
del coupons
del data

In [104]:
data_2[data_2.shopper == 0]

,week,shopper,product,price,discount,no_products_bought,week_basket_size,mean_basket_size,spend,week_basket_value,mean_basekt_value,mean_product_price,discounted,customer_product_discounted_share,customer_product_sales,week_customer_product_sales,mean_customer_product_sales
0,0,0,71,629,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.029851,67,1,0.087013
1,0,0,91,605,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.041096,73,1,0.094805
2,0,0,116,715,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.033333,30,1,0.038961
3,0,0,123,483,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.000000,43,1,0.055844
4,0,0,157,592,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.000000,30,1,0.038961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68076686,89,0,179,415,30.0,770,10,8.992208,451730,5455,5273.981818,586.662338,1,0.125000,8,1,0.010390
68076687,89,0,184,651,0.0,770,10,8.992208,451730,5455,5273.981818,586.662338,0,0.019231,52,1,0.067532
68076688,89,0,211,613,0.0,770,10,8.992208,451730,5455,5273.981818,586.662338,0,0.000000,12,1,0.015584
68076689,89,0,225,602,0.0,770,10,8.992208,451730,5455,5273.981818,586.662338,0,0.029412,68,1,0.088312


In [126]:
data_2[data_2.shopper == 0]['product'].value_counts() #54

91     73
225    68
71     67
130    61
184    52
123    43
202    39
67     35
167    32
116    30
157    30
171    29
87     22
76     17
238    14
249    13
211    12
210    11
214    10
11      9
191     9
52      9
4       8
179     8
244     7
207     6
110     6
38      5
148     5
16      4
56      3
81      3
181     3
98      2
48      2
172     2
80      2
79      2
100     2
15      1
185     1
21      1
37      1
162     1
46      1
138     1
144     1
59      1
243     1
240     1
232     1
122     1
119     1
224     1
Name: product, dtype: int64

In [127]:
data_2[data_2.shopper == 0]['week'].value_counts() # 90 weeks 

51    13
38    12
40    12
7     12
59    12
      ..
46     6
71     5
63     5
14     3
24     3
Name: week, Length: 90, dtype: int64

In [ ]:
# product-price dict -> get prices for products 
product_price_dict = data_2.set_index('product').to_dict()['price'] 
data_3['price']= data_3['product'].map(product_price_dict)

In [119]:
data_2

,week,shopper,product,price,discount,no_products_bought,week_basket_size,mean_basket_size,spend,week_basket_value,mean_basekt_value,mean_product_price,discounted,customer_product_discounted_share,customer_product_sales,week_customer_product_sales,mean_customer_product_sales,no_shopping_events
0,0,0,71,629,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.029851,67,1,0.087013,90
1,0,0,91,605,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.041096,73,1,0.094805,90
2,0,0,116,715,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.033333,30,1,0.038961,90
3,0,0,123,483,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.000000,43,1,0.055844,90
4,0,0,157,592,0.0,770,10,8.992208,451730,5908,5273.981818,586.662338,0,0.000000,30,1,0.038961,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68091869,89,1999,144,562,0.0,799,13,9.322904,457007,7515,5336.280350,571.973717,0,0.039216,51,1,0.063830,90
68091870,89,1999,158,566,0.0,799,13,9.322904,457007,7515,5336.280350,571.973717,0,0.019231,52,1,0.065081,90
68091871,89,1999,192,549,0.0,799,13,9.322904,457007,7515,5336.280350,571.973717,0,0.000000,73,1,0.091364,90
68091872,89,1999,213,592,0.0,799,13,9.322904,457007,7515,5336.280350,571.973717,0,0.000000,17,1,0.021277,90


In [133]:
data_2.discounted.value_counts()/len(data_2)

0    0.960654
1    0.039346
Name: discounted, dtype: float64

In [131]:
lop = data_2[["shopper","week","product"]].sort_values(by=["shopper", "week"]).groupby(by=["shopper","week"]).agg({'product':lambda x: list(x)}).reset_index(drop=False)
lop = lop.rename(columns={"product": "list_of_products"})
# remove duplicates
lop["list_of_products"] = lop["list_of_products"].apply(lambda x: list(dict.fromkeys(x)))
lop.head()

,shopper,week,list_of_products
0,0,0,"[71, 91, 116, 123, 157, 167, 171, 184, 207, 225]"
1,0,1,"[71, 91, 130, 157, 167, 191, 225]"
2,0,2,"[52, 67, 71, 87, 91, 130, 184, 211, 238]"
3,0,3,"[71, 91, 123, 130, 179, 184, 214, 225]"
4,0,4,"[48, 67, 76, 91, 116, 123, 184, 225]"


In [125]:
data_2['week_customer_product_sales'].value_counts()

1    1378720
Name: week_customer_product_sales, dtype: int64

In [60]:
data_2['no_products_bought'] = data_2.groupby(['shopper'])['product'].transform('count')

In [63]:
data_2['week_basket_size'] = data_2.groupby(['week','shopper'])['product'].transform('count')

In [66]:
data_2['mean_basket_size'] = data_2.groupby(['shopper'])['week_basket_size'].transform('mean')

In [68]:
data_2['spend'] = data_2.groupby(['shopper'])['price'].transform('sum')

In [70]:
data_2['week_basket_value'] = data_2.groupby(['week','shopper'])['price'].transform('sum')

In [76]:
data_2['mean_basekt_value'] = data_2.groupby(['shopper'])['week_basket_value'].transform('mean')

In [77]:
data_2['mean_product_price'] = data_2.spend/data_2.no_products_bought

In [79]:
data_2['discounted'] = np.where(data_2['discount']>0,1,0)

In [81]:
data_2['customer_product_discounted_share'] = data_2.groupby(['shopper','product'])['discounted'].transform('mean')

In [83]:
data_2['customer_product_sales'] = data_2.groupby(['shopper','product'])['product'].transform('count')

In [93]:
data_2['week_customer_product_sales'] = data_2.groupby(['week', 'shopper', 'product'])['price'].transform('count')

In [95]:
data_2['mean_customer_product_sales'] = data_2['customer_product_sales']/data_2['no_products_bought']

In [118]:
data_2['no_shopping_events'] = data_2.groupby(['shopper'])['week'].transform('nunique')

## Feature ideas

* products_count: data_2.groupby(['shopper'])['product'].transform.('count')
* basket_size: data_2.groupby(['week','shopper'])['product'].transform.('count')
* mean_basket_size: data_2.groupby(['shopper'])['basket_size'].transform.('mean')
---
* spend: data_2.groupby(['shopper'])['price'].transform.('sum')
* basket_value: data_2.groupby(['week','shopper'])['price'].transform.('sum')
* mean_basket_value: data_2.groupby(['shopper'])['basket_value'].transform.('mean') or mean_basket_value: data_2.spend/data_2.products_count
---
* discounted: data_2["discounted"] = np.where(data_2["discount"] > 0,1,0)
* customer_product_discounted_share: #How often did this customer buy this product at a discounted price? 
---
* how often did customer buy that product? 
* how often did customer buy that product at discounted price?
* how many items of that product did customer buy at specific week? 
* what share of all products bought is that product for customer? 

## DONE!

In [ ]:
data_2['Last_Week_Sales'] = data_2.groupby(['shopper', 'product'])['Sales'].shift()

## TO DO !
--- 
### Creating lag features considering the customer-product combination
* how many weeks since customer bought product the last time, the 2nd last time, 3rd last time, 4th last time? 
    * lag_product_0, lag_product_1, lag_product_2, lag_product_3
* how many weeks on average between buying that product? 
---
* Does shopper buy every week? How many weeks does shopper shop?  At which weeks? 
* 
---
* Demand?/ how often is that product bought on average (whole population) 
* Customer_Demand vs. General_Demand
* 
---
* price elasticity for customer
* price elasticity for customer for that specific product? 
* price elasticity for specific product > price elasticity for customer?
---
* demand for product?
---
* category? 
* is product brought along with complementary product? 
* is product substitued for substitute product? 
* association rules mining/ co-occurance matrix

In [ ]:
# Bracket the weeks
max_week = data_2['week'].max()  
min_week = data_2['week'].min()  
weeks = (max_week - min_week)

# Master list of weeks
week_list = [min_week + datetime.timedelta(weeks=x) for x in range(weeks)]
week_list.sort(reverse=True)

# Master list of shoppers
unique_shoppers = list(set(data_2['shopper'].unique()))
unique_shoppers.sort()
num_unique_shoppers = len(unique_shoppers)

# Unique Week / Shopper index
week_index = np.repeat(weeks, num_unique_shoppers) # num weeks * num shoppers
shopper_index = np.concatenate([unique_shoppers]*days)

row_count = weeks * len(unique_shoppers)

train_test_master = pd.DataFrame(index=range(row_count), columns=all_cols)

train_test_master['week'] = date_index
train_test_master['shopper'] = store_index

# Set the indexes (makes it easy to insert test/train data into master)
train_test_master.set_index(['Date', 'Store'], drop=True, inplace=True)
train.set_index(['Date', 'Store'], drop=True, inplace=True)
test.set_index(['Date', 'Store'], drop=True, inplace=True)

# Update the master index with train and test
train_test_master.update(train)
train_test_master.update(test)

In [ ]:
data['rolling_mean'] = data['Count'].rolling(window=7).mean() # rolling window

![RollingWindowUrl](https://cdn.analyticsvidhya.com/wp-content/uploads/2019/11/3hotmk.gif "Rolling Window")

In [ ]:
data_2['expanding_mean'] = data_2['Count'].expanding(2).mean() # expanding window 

![SegmentLocal](expanding_window.gif "expanding window")

In [110]:
df_lagged = data_2.copy()
trailing_window_size = 10

for window in range(1, trailing_window_size + 1):
    shifted = data_2.shift(window)
    shifted.columns = [x + "_lag" + str(window) for x in data_2.columns]
    
    df_lagged = pd.concat((df_lagged, shifted), axis=1)
df_lagged = df_lagged.dropna()
df_lagged.head()

,week,shopper,product,price,discount,no_products_bought,week_basket_size,mean_basket_size,spend,week_basket_value,...,mean_basket_size_lag10,spend_lag10,week_basket_value_lag10,mean_basekt_value_lag10,mean_product_price_lag10,discounted_lag10,customer_product_discounted_share_lag10,customer_product_sales_lag10,week_customer_product_sales_lag10,mean_customer_product_sales_lag10
10,0,1,22,528,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.029851,67.0,1.0,0.087013
11,0,1,35,602,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.041096,73.0,1.0,0.094805
12,0,1,92,543,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.033333,30.0,1.0,0.038961
13,0,1,146,596,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.000000,43.0,1.0,0.055844
14,0,1,168,588,0.0,665,8,8.016541,388503,4648,...,8.992208,451730.0,5908.0,5273.981818,586.662338,0.0,0.000000,30.0,1.0,0.038961


In [ ]:
def lag_by_group(key, value_df):
    df = value_df.assign(group = key) # this pandas method returns a copy of the df, with group columns assigned the key value
    return (df.sort_values(by=["date"], ascending=True)
        .set_index(["date"])
        .shift(1)
               ) # the parenthesis allow you to chain methods and avoid intermediate variable assignment


In [ ]:
dflist = [lag_by_group(g, grouped_df.get_group(g)) for g in grouped_df.groups.keys()]
pd.concat(dflist, axis=0).reset_index())

In [124]:
#import numpy as np

def init_nans(df, num_periods, lag_vars, col_name_suf):
    '''
    Initiate column(s) of nan values for your lag columns in the given dataframe
    Inputs:
        df - dataframe
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        lag_vars - columns of values to create lag columns with
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
    '''
    for period in range(1, num_periods + 1):
        for lag_var in lag_vars:
            df['{}_lag{}{}'.format(lag_var, period, col_name_suf)] = np.nan

def set_lag_vals(df, comb_mask, date_col, lag_vars, num_periods, col_name_suf):
    '''
    Sets lag values according to the last column in the heirarchy or columns
    Inputs:
        df - dataframe
        comb_mask - combined mask for current value combinations between columns. Used to filter dataframe
        date_col - date column to use in grouping and lag periods
        lag_vars - columns of values to create lag columns with
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
    '''
    foo = df[ comb_mask ].sort_values(date_col, ascending=False)     
    length = len(foo[date_col].unique()) # determine number of visits (because multiple item categories can be updated in a single visit)
    for period in range(1, num_periods + 1):
        # skip if there's not enough data to create lag columns
        if length < period + 1:
            continue
        i = 0
        # create duplicate df, but with all indices shifted by the current 'period' number
        foo_shifted = foo.shift(-period)
        foo_grouped = foo.groupby(date_col).mean()
        for index, row in foo.iterrows():
            date = foo_shifted[ foo_shifted.index == index ][date_col].values[0]
            for lag_var in lag_vars:
                lag_val = foo_grouped[ foo_grouped.index == date ][lag_var].values[0]
                # set value
                df.set_value(index, '{}_lag{}{}'.format(lag_var, period, col_name_suf), lag_val)
            i += 1
            if i + period == length:
                break # back to period loop

def lag_rec(df, num_periods, col_filters, date_col, lag_vars, col_name_suf, mask=True):
    '''
    Recursively loop through various heirarchaly ordered columns, grouping by date
    INPUTS:
        df - pandas dataframe
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        col_filters - columns to heiracrchally filter down on, with the last column being the one ultimately used
        date_col - date column to use in grouping and lag periods
        lag_vars - columns of values to create lag columns with
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
        mask - DO NOT CHANGE. Required to be True to maintain dataframe mask between recursive iterations
    '''
    # begin with mask of all trues
    true_mask = np.ones(len(df), dtype=bool)
    loop_mask = mask & true_mask
    col_filter = col_filters[0]
    for val in df[ loop_mask ][col_filter].unique():
        val_mask = df[col_filter] == val
        comb_mask = loop_mask & val_mask

        if len(col_filters) > 1:
            #recursively update the remaining items' positions
            lag_rec(df, num_periods, col_filters[1:], date_col, lag_vars, col_name_suf, mask=comb_mask)
        else:
            set_lag_vals(df, comb_mask, date_col, lag_vars, num_periods, col_name_suf)

def lag(df, num_periods, col_filters, date_col, lag_vars, col_name_suf):
    '''
    Wrapper function to execute init_nans and lag_rec.
    Inputs:
        df - dataframe
        num_periods - number of periods (previous dates) to go back and attempt to fill lag values for
        col_filters - columns to heiracrchally filter down on, with the last column being the one ultimately used
        date_col - date column to use in grouping and lag periods
        lag_vars - columns of values to create lag columns with
        col_name_suf - suffix to append to newly created columns (help to distinguish between last filtered column choosen)
    Returns:
        Updated dataframe containing new lag columns
    '''
    init_nans(df, num_periods, lag_vars, col_name_suf)
    lag_rec(df, num_periods, col_filters, date_filter, lag_vars, col_name_suf)
    return df

# Problems

##### Products are not in basket every week. But pred_idx has all products listed for week 90 for each shopper
* What to do with products which are not bought each week? 
    * Build training data set including products which are not bougth 
    * Build feature "sold" (1/0) 
    * Derive product price from list 